In [1]:
from einops import rearrange
import copy
import h5py
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from pdb import set_trace
import matplotlib.pyplot as plt
from torch import nn
from x_transformers import  Encoder, Decoder
from x_transformers.autoregressive_wrapper import exists
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
from fastai.vision.all import BCEWithLogitsLossFlat
from transformers.optimization import (
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup,
)
from fastprogress.fastprogress import master_bar, progress_bar
import os
from timm import create_model
from timm.data.mixup import Mixup

In [2]:
class CFG:
    bs = 96
    nw = 4
    model_name = "convnext_large_in22k"
    lr = 1e-4
    wd = 1e-4
    epoch = 10
    warmup_pct = 0.1
    num_classes = 1
    dropout_rate = 0.3
    folder = "EXP_120_00_VIT_36_MIXUP_RANDOM_SAMPLER"
    mixup=False
    exp_name = f"{folder}_{model_name}"

In [3]:
def get_snr(left, right, df):
    df_ = pd.concat([df.query(f"snr>{left} & snr<{right}"), df.query("snr==0")])
    return df_


def generate_report(df, p, fn):
    pred = torch.sigmoid(p).cpu().numpy().reshape(-1)
    val_df_eval = df.copy()
    val_df_eval["pred"] = pred
    val_df_eval.to_csv(f"{fn}_oof.csv")

    roc_100 = roc_auc_score(val_df_eval["target"], val_df_eval["pred"])
    
    tr_comp = val_df_eval.query('data_type=="comp_train"')
    roc_comp_train = roc_auc_score(tr_comp['target'], tr_comp['pred'])
    roc_0_50 = roc_auc_score(
        get_snr(0, 50, val_df_eval)["target"], get_snr(0, 50, val_df_eval)["pred"]
    )
    roc_15_50 = roc_auc_score(
        get_snr(15, 50, val_df_eval)["target"], get_snr(15, 50, val_df_eval)["pred"]
    )
    roc_25_50 = roc_auc_score(
        get_snr(25, 50, val_df_eval)["target"], get_snr(25, 50, val_df_eval)["pred"]
    )
    roc_0_40 = roc_auc_score(
        get_snr(0, 40, val_df_eval)["target"], get_snr(0, 40, val_df_eval)["pred"]
    )

    roc_0_30 = roc_auc_score(
        get_snr(0, 30, val_df_eval)["target"], get_snr(0, 30, val_df_eval)["pred"]
    )

    return {
        "roc_all": roc_100,
        "roc_0_50": roc_0_50,
        "roc_15_50": roc_15_50,
        "roc_25_50": roc_25_50,
        "roc_0_40": roc_0_40,
        "roc_0_30": roc_0_30,
        "roc_comp_train": roc_comp_train
    }

class SaveModel:
    def __init__(self, folder, exp_name, best=np.inf):
        self.best = best
        self.folder = Path(folder) / f"{exp_name}.pth"

    def __call__(self, score, model, epoch):
        if score < self.best:
            self.best = score
            print(f"Better model found at epoch {epoch} with value: {self.best}.")
            torch.save(model.state_dict(), self.folder)


class SaveModelMetric:
    def __init__(self, folder, exp_name, best=-np.inf):
        self.best = best
        self.folder = Path(folder) / f"{exp_name}.pth"

    def __call__(self, score, model, epoch):
        if score > self.best:
            self.best = score
            print(f"Better model found at epoch {epoch} with value: {self.best}.")
            torch.save(model.state_dict(), self.folder)


class SaveModelEpoch:
    def __init__(self, folder, exp_name, best=-np.inf):
        self.best = best
        self.folder = Path(folder)
        self.exp_name = exp_name

    def __call__(self, score, model, epoch):
        self.best = score
        print(f"Better model found at epoch {epoch} with value: {self.best}.")
        torch.save(model.state_dict(), f"{self.folder/self.exp_name}_{epoch}.pth")


def custom_auc_score(p, gt):
    return roc_auc_score(gt.cpu().numpy(),  torch.sigmoid(p).cpu().numpy().reshape(-1))


def fit_mixup(
    epochs,
    model,
    train_dl,
    valid_dl,
    loss_fn,
    opt,
    metric,
    val_df,
    folder="models",
    exp_name="exp_00",
    device=None,
    sched=None,
    mixup_=False,
    save_md=SaveModelMetric,
):
    if device is None:
        device = (
            torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        )

    os.makedirs(folder, exist_ok=True)
    loss_fn_trn = loss_fn
    if mixup_:
        mixup = Mixup(num_classes=2, mixup_alpha=0.4, prob=0.2, label_smoothing=0)
        print('doing mixup')
    mb = master_bar(range(epochs))

    mb.write(
        [
            "epoch",
            "train_loss",
            "valid_loss",
            "val_metric",
            "roc_all",
            "roc_0_50",
            "roc_15_50",
            "roc_25_50",
            "roc_0_40",
            "roc_0_30",
            "roc_comp_train",
        ],
        table=True,
    )
    model.to(device)  # we have to put our model on gpu
    #scaler = torch.cuda.amp.GradScaler()  # this for half precision training
    save_md = save_md(folder, exp_name)

    for i in mb:  # iterating  epoch
        trn_loss, val_loss = 0.0, 0.0
        trn_n, val_n = len(train_dl.dataset), len(valid_dl.dataset)
        model.train()  # set model for training
        for (xb, yb) in progress_bar(train_dl, parent=mb):
            xb, yb = xb.to(device), yb.to(device)  # putting batches to device
            if mixup_:
                xb, yb = mixup(xb, yb)
                yb = (yb[:, 0] > 0).float()
           
            out = model(xb)  # forward pass
            loss = loss_fn_trn(out, yb)  # calulation loss

            trn_loss += loss.item()
            #print(loss.item())
            opt.zero_grad()  # zeroing optimizer
            loss.backward()  # backward
            opt.step()  # optimzers step
            if sched is not None:
                sched.step()  # scuedular step

        trn_loss /= mb.child.total

        # putting model in eval mode
        model.eval()
        gt = []
        pred = []
        # after epooch is done we can run a validation dataloder and see how are doing
        with torch.no_grad():
            for (xb, yb) in progress_bar(valid_dl, parent=mb):
                xb, yb = xb.to(device), yb.to(device)
                out = model(xb)
                loss = loss_fn(out, yb)
                val_loss += loss.item()

                gt.append(yb.detach())
                pred.append(out.detach())
        # calculating metric
        metric_ = metric(torch.cat(pred), torch.cat(gt))
        # saving model if necessary
        save_md(metric_, model, i)
        val_loss /= mb.child.total
        dict_res = generate_report(val_df, torch.cat(pred), f"{folder}/{exp_name}_{i}")

        pd.DataFrame(
            {
                "trn_loss": [trn_loss],
                "val_loss": [val_loss],
                "metric": [metric_],
                "roc_all": [dict_res["roc_all"]],
                "roc_0_50": [dict_res["roc_0_50"]],
                "roc_15_50": [dict_res["roc_15_50"]],
                "roc_25_50": [dict_res["roc_25_50"]],
                "roc_0_40": [dict_res["roc_0_40"]],
                "roc_0_30": [dict_res["roc_0_30"]],
                "roc_comp_train": [dict_res["roc_comp_train"]]
            }
        ).to_csv(f"{folder}/{exp_name}_{i}.csv", index=False)
        mb.write(
            [
                i,
                f"{trn_loss:.6f}",
                f"{val_loss:.6f}",
                f"{metric_:.6f}",
                f"{dict_res['roc_all']:.6f}",
                f"{dict_res['roc_0_50']:.6f}",
                f"{dict_res['roc_15_50']:.6f}",
                f"{dict_res['roc_25_50']:.6f}",
                f"{dict_res['roc_0_40']:.6f}",
                f"{dict_res['roc_0_30']:.6f}",
                f"{dict_res['roc_comp_train']:.6f}",
            ],
            table=True,
        )
    print("Training done")
    # loading the best checkpoint

In [4]:
import random
class DataV0():
    """
    dataset = Dataset(data_type, df)

    img, y = dataset[i]
      img (np.float32): 2 x 360 x 128
      y (np.float32): label 0 or 1
    """
    def __init__(self, df, freq_tfms=False, iterations = None):
        self.df = df
        self.tfms = freq_tfms
        self.iterations = iterations
        

    def __len__(self):
        if self.iterations:
            return self.iterations
        return len(self.df)

    def __getitem__(self, i):
        """
        i (int): get ith data
        """
        if self.iterations:
            i = np.random.randint(0, len(self.df))
            r = self.df.iloc[i]
        else:
            r = self.df.iloc[i]
        y = np.float32(r.target)
        img = np.array(torch.load(r.id)['s_p_n'])
        
        if self.tfms:
            if np.random.rand() <= 0.5:  # horizontal flip
                img = np.flip(img, axis=1).copy()
            if np.random.rand() <= 0.5:  # vertical flip
                img = np.flip(img, axis=2).copy()
            if np.random.rand() <= 0.5:  # vertical shift
                img = np.roll(img, np.random.randint(low=0, high=img.shape[1]), axis=1)
        return torch.tensor(img), y

In [5]:
trn_df = pd.read_csv('../data/SPLITS/V_20/trn_df.csv')
trn_df['id'] = trn_df['id'].apply(lambda x: Path(x.replace('.h5', '.pth')))
print(len(trn_df))
val_df = pd.read_csv('../data/SPLITS/V_20/val_df.csv')


comp_train = pd.read_csv('../data/train_labels.csv')
comp_train.columns = ['fn', 'target']
comp_train = comp_train.query('target>=0')
comp_train['fn'] = comp_train['fn'].apply(lambda x: Path('../data/train')/f'{x}.pth')
comp_train.columns = ['id', 'target']
comp_train['data_type'] = 'comp_train'

comp_noise = pd.read_csv('../data/custom_data/DATA_V33/train.csv')
comp_noise['id'] = comp_noise['id'].apply(lambda x: Path(x.replace('.h5', '.pth')))

val_df = pd.concat([val_df, comp_train, comp_noise[:400]], ignore_index=True)
trn_df = pd.concat([trn_df, comp_noise[400:]], ignore_index=True)
print(len(trn_df))

16252
17350


In [6]:
# Train - val split
fold =0
trn_ds = DataV0(trn_df, True)
vld_ds = DataV0(val_df)

trn_dl = DataLoader(
    trn_ds,
    batch_size=CFG.bs,
    shuffle=True,
    num_workers=CFG.nw,
    pin_memory=True,
    drop_last=True,
)
vld_dl = DataLoader(
    vld_ds,
    batch_size=CFG.bs,
    shuffle=False,
    num_workers=CFG.nw,
    pin_memory=True,
)

custom_model = create_model(
                    CFG.model_name,
                    pretrained=True,
                    num_classes=1,
                    in_chans=2,
                )

opt = torch.optim.AdamW(custom_model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
loss_func = BCEWithLogitsLossFlat()
warmup_steps = int(len(trn_dl) * int(CFG.warmup_pct * CFG.epoch))
total_steps = int(len(trn_dl) * CFG.epoch)
sched = get_linear_schedule_with_warmup(
    opt, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

fit_mixup(
    epochs=CFG.epoch,
    model=custom_model,
    train_dl=trn_dl,
    valid_dl=vld_dl,
    loss_fn=loss_func,
    opt=opt,
    val_df=val_df,
    metric=custom_auc_score,
    folder=CFG.folder,
    exp_name=f"{CFG.exp_name}_{fold}",
    device="cuda:0",
    sched=sched,
    mixup_=False,
)

epoch,train_loss,valid_loss,val_metric,roc_all,roc_0_50,roc_15_50,roc_25_50,roc_0_40,roc_0_30,roc_comp_train
0,0.445388,0.398437,0.871810,0.871810,0.691610,0.691610,0.699937,0.590207,0.530123,0.789281
1,0.354223,0.389733,0.893639,0.893639,0.739201,0.739201,0.753215,0.634874,0.578111,0.811037
2,0.319456,0.362075,0.900515,0.900515,0.754179,0.754179,0.767279,0.645239,0.578514,0.809587
3,0.304229,0.346776,0.909271,0.909271,0.773548,0.773548,0.787275,0.664659,0.580506,0.820938
4,0.292073,0.348600,0.910873,0.910873,0.780349,0.780349,0.794042,0.668606,0.575206,0.814062
5,0.286270,0.341194,0.911979,0.911979,0.785361,0.785361,0.798724,0.679606,0.607568,0.819000
6,0.274581,0.350865,0.912992,0.912992,0.774762,0.774762,0.791433,0.663265,0.573185,0.823050
7,0.263425,0.366122,0.909568,0.909568,0.767966,0.767966,0.785464,0.658354,0.581132,0.813050
8,0.259145,0.351964,0.910919,0.910919,0.773903,0.773903,0.788785,0.664204,0.589432,0.815613
9,0.246426,0.374769,0.910328,0.910328,0.771262,0.771262,0.786956,0.661980,0.578136,0.814987


Better model found at epoch 0 with value: 0.8718098997493735.
Better model found at epoch 1 with value: 0.8936385964912281.
Better model found at epoch 2 with value: 0.9005145363408522.
Better model found at epoch 3 with value: 0.9092711779448622.
Better model found at epoch 4 with value: 0.9108734335839599.
Better model found at epoch 5 with value: 0.9119794486215539.
Better model found at epoch 6 with value: 0.9129924812030076.
Training done
